Notes: 
* Answer each question in a separate Jupynet Notebook Cell
* Pleas keep the code in your cells short. 
  * In notebook programming cells are typicaly short to facilitate reading. 
  * If well toughout, most answers in this assignment won're require more than 3 or 4 lines of code. 
* Do no change the list of import, i.e., do not add additional libraries. Those included are the only ones you are allowed to use.
* Add your first and Last name below:

# Name: Justin Honda
<Justin> <Honda>
    


In [1]:
import pandas as pd
from itertools import product
from collections import Counter
from tqdm.notebook import tqdm
import random
import json

In this assignment you will be working with Corona Virus (SARS-CoV2) data that was obtained from the [National Center for Biotechnology Information](https://www.ncbi.nlm.nih.gov/). You will need two files. The first (`data/coronavirus_info.csv`) is small and is provided in the GitHub Repo. The second  (`data_report.jsonl`) is larger so you will need to download a compressed version, which you will need to uncompress prior to using. You can downlod the second file here:

https://www.dropbox.com/s/qdn67rshygz06ff/data_report.jsonl.gz?dl=0

We start by loading `data/coronavirus_info.csv` (Code provided below)

In [2]:
# CODE PROVIDED -- DO NOT REMOVE
table = pd.read_csv("data/coronavirus_info.csv", low_memory=False)
table = table.drop(["US State", "Host Name", "Host Taxonomy ID", "Sequence Type", "Species Taxonomy Id", "Nuc Completeness", "BioProject", "BioSample"], axis=1)

missing = table["Geo Location"].isnull()
table.loc[missing, "Geo Location"] = ""


table.head(10)

,Nucleotide Accession,Species Name,Virus Genus,Virus Family,Isolate Name,Nucleotide Length,Geo Location,Collection Date
0,NC_045512.2,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,Wuhan-Hu-1,29903,Asia; China,2019-12
1,OK058807.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04825/2021,29801,North America; USA,2021-07-29
2,OK058777.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04790/2021,29771,North America; USA,2021-08-10
3,OK058695.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04700/2021,29820,North America; USA,2021-08-15
4,OK058662.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04651/2021,29798,North America; USA,2021-08-09
5,OK058592.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/MA-MASPHL-04499/2021,29802,North America; USA,2021-07-26
6,OK056996.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/FL-CDC-QDX27934346/2021,29775,North America; USA: Florida,2021-08-18
7,OK056909.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/CA-CDC-QDX27909662/2021,29775,North America; USA: California,2021-08-16
8,OK056850.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/FL-CDC-QDX27934406/2021,29763,North America; USA: Florida,2021-08-18
9,OK056784.1,Severe acute respiratory syndrome coronavirus 2,Betacoronavirus,Coronaviridae,SARS-CoV-2/human/USA/NY-CDC-QDX28007789/2021,29775,North America; USA: New York,2021-08-21


### Q.1

* The location of each of the sequences is recorded under the `Geo Location` column.  How many entries are from Asia?
  * Note that for some records, the `Geo Location` column is missing
  * Display the results using the following format: 
    Asia: XXXX,
    North America': XXXX,
    Europe: XXXX,
    Oceania: XXXX,
    Africa: XXXX,
    South America: XXXX 


In [3]:
num_asia = len(table[table['Geo Location'].str.contains('Asia')]['Geo Location'].to_numpy()) #3903
num_NA = len(table[table['Geo Location'].str.contains('North America')]['Geo Location'].to_numpy()) #224014
num_eur = len(table[table['Geo Location'].str.contains('Europe')]['Geo Location'].to_numpy()) #189100
num_ocea = len(table[table['Geo Location'].str.contains('Oceania')]['Geo Location'].to_numpy()) #10301
num_afr = len(table[table['Geo Location'].str.contains('Africa')]['Geo Location'].to_numpy()) #1405
num_SA = len(table[table['Geo Location'].str.contains('South America')]['Geo Location'].to_numpy()) #534

print(f'Asia: {num_asia}, North America: {num_NA}, Europe: {num_eur}, Oceania: {num_ocea}, Africa: {num_afr}, South America: {num_SA}')

Asia: 3903, North America: 224014, Europe: 189100, Oceania: 10301, Africa: 1405, South America: 534


### Q.2
Use the `coronavirus_info.csv` table to count the entries that are from Hawaii. 

In [4]:
num_hawaii = len(table[table['Geo Location'].str.contains('Hawaii')]['Geo Location'].to_numpy())
print(f'Hawaii: {num_hawaii}') #119

Hawaii: 119


### Q.3

The file `data_report.jsonl` contains the variants of the virus in the DB. This `json` file is a list of records (one per line) for each one of the genomes in the database. Before we work with the large file, we will experiment with a file containing a single record.

The file `single_record.json` contains a single sample record. Use the `JSON library to load the file `single_record.json` into a variable called `sample_vir_record`

1. how many first-level keys does this record have?
  * Do not count nested keys. Only those at the top level should be counted



In [5]:
with open('data/single_record.json') as f:
    sample_vir_record = json.load(f)
    
num_keys = len((sample_vir_record.keys()))
print(f'There are {num_keys} first-level keys in the record.')

There are 18 first-level keys in the record.


### Q.4

Each Covid various in this database is classified according to a system referred to as the Pangolin (Phylogenetic Assignment of Named Global Outbreak LINeages) classification. It is not essential to complete the assignment that you understand this system, but if you're interested in learning more, see:

https://cov-lineages.org/resources/pangolin.html

The Pangolin classification of this sample record is nested within the `virus` key:
```json
{ ...
  "virus": {
              ...
              "pangolinClassification": 
              ...
            }
  ... 
}
```
Write code to extract the classification of this record. The result should be `B.1.1.214`

In [6]:
sample_vir_record['virus']['pangolinClassification']

'B.1.1.214'

We hear in the news about `Alpha`, `Beta`, `Delta` variants of concern and recently the Mu variant as being of interest. Basedon your answer to `Q.3`, you may have been tempted to infer that this virus is of type `Beta` since the first letter is `B`. In fact, this variant of type `Alpha` and is a variant of concern. Although not relevant to this exercise, the rules for naming new variants and the list of known `SARS-CoV-2` are provided here:

https://www.pango.network/how-does-the-system-work/what-are-pango-lineages/

https://cov-lineages.org/lineage_list.html


The following short video is very helpful for understanding what a variant is, how it arises, how it's named, and why some variants are more concerning than others.

https://www.youtube.com/watch?v=B8UEZ9cfgz4

### Q.5

Write Python code that counts all the different kinds of variants in `data_report.jsonl`. 
Because the file is 7GB, you won't likely be able to load it into your laptop's RAM using Python. I encountered this error when trying to open it on my laptop

![](https://www.dropbox.com/s/lieo685pafkgm5e/ram_error.png?dl=1)


It would be easy to extract the data from the pangolinClassification field of each `json` record by reading each line (a record) at a time.

The list of current variants of concern we are interested in are:
      * Alpha (B.1.1.7)
      * Beta (B.1.351, B.1.351.2, B.1.351.3)
      * Delta (B.1.617.2, AY.1, AY.2, AY.3)
      * Gamma (P.1, P.1.1, P.1.2) 

You should get something similar to what follows:
```
Alpha: X
Beta: X
Delta: X
Gamma: X
```
Where `X` represents the counts for relevant variants

In [7]:
info = open('data/data_report.jsonl')
alpha_count = 0
beta_count = 0
delta_count = 0
gamma_count = 0
for line in info:
    temp = json.loads(line)
    if('pangolinClassification' in temp['virus']):
        if temp['virus']['pangolinClassification'] == "B.1.1.7":
            alpha_count += 1
        elif temp['virus']['pangolinClassification'] == "B.1.351" or "B.1.351.2" or "B.1.351.3":
            beta_count += 1
        elif temp['virus']['pangolinClassification'] == "B.1.617.2" or "AY.1" or "AY.2" or "AY.3":
            delta_count += 1
        elif temp['virus']['pangolinClassification'] == "P.1" or "P.1.1" or "P.1.2":
            gamma_count += 1
print(f'Alpha: {alpha_count}\nBeta: {beta_count}\nDelta: {delta_count}\nGamma: {gamma_count}')

Alpha: 178423
Beta: 250648
Delta: 0
Gamma: 0


# Find similar viruses.

It's often useful to compare viruses to study how similar strains are. While sophisticated algorithms to compare a pair of viruses exist, these are typically computationally intensive and cannot be used to carry out a large number of comparisons. 

An alternative, albeit less sensitive, approach consists of comparing word counts (called k-mers, where k is the word size) across genomes.  Suppose we have two viruses X and Y, with the following Genomes.
```
X = "ACGTAGTGCATGTGTAGCTGTGTAGCTGTAC"
Y = "ACTAGTGCATGTGTAGCTCTGTAGCTGATAC"
```

To compare `X` and `Y`, we first vectorize these genomes by marking the presence of words (k-mers) as a boolean value, 0 if absent and 1 if the word is present. This method assumes that similar genomes will have the same words, which makes sense.

This idea, which is referred to as the bag of words model is computationally efficient, making it ideal to vectorize text in big data analytics. Another variant of this model requires replacing the presence and absence by counts for each word.

The code below vectorizes an input DNA sequence intro k-mers of size k=2

In [8]:
# CODE PROVIDED -- DO NOT REMOVE
def get_kmer_2(X):
    DNA = ["A", "C", "G", "T"]
    words_size_2 = ["".join(dna_prod) for dna_prod in product(DNA, DNA)]
    counts = pd.Series([0 for _ in words_size_2], index = words_size_2)
    words_in_X = set([X[i:i+2] for i in range(0, len(X)-1)])
    counts[list(words_in_X)] = 1
    return counts    

In [9]:
# CODE PROVIDED -- DO NOT REMOVE
# X has 3 words of size 2 (AC, CG, GT)
X = "ACGT"
get_kmer_2("ACGT")

AA    0
AC    1
AG    0
AT    0
CA    0
CC    0
CG    1
CT    0
GA    0
GC    0
GG    0
GT    1
TA    0
TC    0
TG    0
TT    0
dtype: int64

The function below takes a dictionary of sequences' counts as a `pandas Series` and prints it using HTML Table, which you might agree is nicer to visualize.

In [10]:
# CODE PROVIDED -- DO NOT REMOVE
def pretty_print_counts(counts_dict):
    list_of_count = [data.to_list() for data in counts_dict.values()]
    list_of_indices = [x for x in counts_dict.keys()]
    list_of_columns = list(counts_dict.values())[0].index.to_list()
    df_single_level_cols = pd.DataFrame(list_of_count,
                                        index=[x for x in counts_dict.keys()],
                                       columns = list_of_columns)    
    return df_single_level_cols 



In [11]:
# CODE PROVIDED -- DO NOT REMOVE

X = "ACGTACGTACGTACGT"
Y = "ACGTACAAACGTACGT"
Z = "TTTTACAAACGTTTTT"

counts_dict = {"X": get_kmer_2(X), "Y": get_kmer_2(Y), "Z": get_kmer_2(Z)}
pretty_print_counts(counts_dict)


,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
X,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0
Y,1,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0
Z,1,1,0,0,1,0,1,0,0,0,0,1,1,0,0,1


### Q.6

Write a function that computes the Jaccard similarity between two feature vectors, A and B. If you recall, Jaccard similarity is computed as:

$$J(A,B) = \frac{A \cap B}{A \cup B}$$

In other words, the number of items shared by `A` and `B` over the set of all items in `A` or `B`.

For example, for `A= get_kmer_2(X)` and Y = get_kmer_2(B) above,

$$
J(A,B) = \frac{4}{6}
$$

Your function should have the following signature: 

`jaccard(A, B)`

Where `A` and `B` are `pandas Series`


Test your function using the code below to make sure it's correct.

In [12]:
def jaccard(A, B):
    intersect = 0
    union = 0
    for i in A.keys():
        if A[i] == 1 and B[i] == 1:
            intersect += 1
            union += 1
        elif A[i] == 1 or B[i] == 1:
            union += 1
    return intersect/union

In [13]:
# TEST PROVIDED -- DO NOT REMOVE
A = get_kmer_2(X)
B = get_kmer_2(Y)
assert jaccard(A, B) == 4/6

### Q.7 

* Compute the jaccard similarity for the pairs of sequences `(X, Y)`, `(X, Z)`, `(Y, Z)`. 


In [14]:
A = get_kmer_2(X) #X
B = get_kmer_2(Y) #Y
C = get_kmer_2(Z) #Z
x_y = jaccard(A, B)
x_z = jaccard(A, C)
y_z = jaccard(B, C)
print(f'Jaccard similarities:\n(X, Y): {x_y}\n(X, Z): {x_z}\n(Y, Z): {y_z}')

Jaccard similarities:
(X, Y): 0.6666666666666666
(X, Z): 0.5714285714285714
(Y, Z): 0.8571428571428571


### Q.8

The vectors representing the presence and absence of words in both `Y` and `Z` are very similar (Jaccard = 0.85), despite major differences at the DNA level between these two sequences. This is because the words are small -- it is as if you were comparing a history book with a book on Python using words of size 2. It's very likely that both books will contain the same words of size 2. Increasing the size of `k` will produce substantial differences. 

Change the function `get_kmer_2` so that given a sequence `X` and a k-mer size `k`, the function returns a boolean vector of all the words of size `k` in `X`. Cal the function `get_kmer`



The following code can be used to generate all DNA words of size `k`
```pyton
words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]
```

Once done, use the code below to test your function

In [116]:
def get_kmer(X, k):
    DNA = ["A", "C", "G", "T"]
    words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]
    counts = pd.Series([0 for _ in words_size_k], index = words_size_k)
    words_in_X = set([X[i:i+k] for i in range(0, len(X)-(k-1))])
    counts[list(words_in_X)] = 1
    return counts  

In [16]:
# TEST PROVIDED -- DO NOT REMOVE
X = "ACGTGATGATTG"

counts = get_kmer(X, k=1)
assert counts.tolist() == [1,1,1,1]


counts = get_kmer(X, k=3)
assert (counts[["ACG", "CGT", "GTG", "TGA", "GAT", "ATG", "ATT", "TTG"]] == 1).sum()  == 8
assert (counts.drop(["ACG", "CGT", "GTG", "TGA", "GAT", "ATG", "ATT", "TTG"]) == 0).sum()  == 56


### Q.9

* Compute the Jaccard similarity for the pairs `(X, Y)`, `(X, Z)`, `(Y, Z)` using `k= 5`


In [115]:
A = get_kmer(X, 5) #X
B = get_kmer(Y, 5) #Y
C = get_kmer(Z, 5) #Z
x_y = jaccard(A, B)
x_z = jaccard(A, C)
y_z = jaccard(B, C)
print(f'Jaccard similarities:\n(X, Y): {x_y}\n(X, Z): {x_z}\n(Y, Z): {y_z}')

Jaccard similarities:
(X, Y): 1.0
(X, Z): 0.375
(Y, Z): 0.375


The appropriate word size varies with the length of the text, with larger words depicting similarity more accurately. However, large values of `k` are:
1. More computationally intensive to compute. With k = 12, there are $4^12 = 16,777,216$ words to compute for each sequence.

2. More likely to skew the distance between fairly similar sequences. For example `k=8`, the Jaccard index between `X` and `Y` is `0`, even though `X` and `Y` have only two mismatching characters. While this is an extreme case due to the fact that X and Y are short, the logic applies to longer sequences and larger values of `k`


![](https://www.dropbox.com/s/rhw5szbiohsqu7w/mismatches.png?dl=1)


In [118]:
# Testing Dr. Belcaid's claim above as mentioned on Slack:
A = get_kmer(X, 8) #X
B = get_kmer(Y, 8) #Y
C = get_kmer(Z, 8) #Z
x_y = jaccard(A, B)
x_z = jaccard(A, C)
y_z = jaccard(B, C)
print(f'Jaccard similarities with k = 8:\n(X, Y): {x_y}\n(X, Z): {x_z}\n(Y, Z): {y_z}')

Jaccard similarities with k = 8:
(X, Y): 1.0
(X, Z): 0.0
(Y, Z): 0.0


The code I used is provided as a reference below. The code took 7 hours to complete on a single machine and approximately 12 minutes on a larger server with 72 cores and 1TB of RAM. To parallelize the execution, I split the file into files that contain 1000 sequences each and used GNU Parallel to run each file on a single CPU core.

In [26]:
# CODE PROVIDED FOR ILLUTRATION -- DO NOT REMOVE
# RUNNING LOCALLY MAY TAKE A LONG TIME TO COMPLETE

# k = 8 
# DNA = ["A", "C", "G", "T"]
# words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]

    
# def get_kmer_mod(X):
#     counts = pd.Series([0 for _ in words_size_k], index = words_size_k)
#     words_in_X = set([X[i:i+k] for i in range(0, len(X)-k+1)])
#     counts[list(words_in_X)] = 1
#     return counts   

# def replace_bad_nucs(seq):
#     for character in ['W', 'K', "Y", "M", 'H']:
#         seq = seq.replace(character, 'A') 
        
#     for character in ['R', 'S', 'D', "V", "B"]:
#         seq = seq.replace(character, 'C') 
        
#     seq = seq.replace("N", '') 
    
#     return seq

# all_counts = []
# all_names = []
# with tqdm(total=1000) as pbar:
#     for record in SeqIO.parse("myseq0.fa", 'fasta'):
#         all_names.append(record.id)
#         seq = replace_bad_nucs(str(record.seq))

#         counts = get_kmer_mod(seq)
#         all_counts.append(counts)
#         pbar.update(1)
    
# kmer_counts = pd.DataFrame(all_counts, index = all_names)
# kmer_counts.head()

### Hashing Sequences

We are interested in finding pairs of sequences that are very similar. However, comparing the sequences pairwise is not tractable since it would require carrying out $429282 * (429282 - 1) / 2 = 92_141_303_121$ comparisons.

Instead, we will use the hashing-based approach covered in class. Rather than hashing a sequence over all k-mers, we will only compute the hash for a subset of k-mers. we will repeat the operation n times to avoid that similar sequences are assigned to different bins due to a single, rare mismatch.

This, as discussed in class, is computationally more efficient compared to computing all pairwise sequences. 

### Q.10 

Write a function that takes a `pandas  Series` and a subset of columns and returns the hash computed on the subset of columns. Call this function`hash_on_subset`.

As an example, consider all words with a size of 2 as follows 

|	|AA	|AC	|AG	|AT	|CA | CC| CG| CT| GA| GC| GG| GT| TA| TC| TG| TT|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| A	|0	|1	|0	|0	|0	|0	|1	|0	| 0 |0	|0	| 1 |1  |0	|0	|0  |

```python
hash_on_subset(A, ["AC", "CG", "CT", "GT", "TA"]) 
```

is equivalent to:

```python
hash((1, 1, 0, 1, 1)) == 5085477689562523216
```


In [19]:
def hash_on_subset(A, subset):
    subset_values = []
    for item in subset:
        subset_values.append(A[item])
    #  subset_tuple = [A[x] for x in subset_values]
    subset_tuple = tuple(subset_values)
    return hash(subset_tuple)


The method `sample` from the random module `m` words from a list

For example, running:
```python
random.sample( ["A", "C", "G", "T"], 2 )
```
returns
```
['A', 'C']
```
The returned subset may be different for you.

* The code below randomly selects `m=20` k-mers we will use to compare the genomes



In [20]:
random.sample( ["A", "C", "G", "T"], 2 )

['C', 'A']

In [22]:
k=8
DNA = ["A", "C", "G", "T"]
words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]

m=20
subset_kmers = random.sample(words_size_k, m)

### Q.12


Apply the function `hash_subset` to all the rows of `all_kmers_df`. The data science (*vectorized*) way to do so is using the `apply` method available on a `pandas DataFrame` instead of using for loops. For example, given a DataFrame `df` such that:

```
df = pd.DataFrame([[1,2,3], [4,5,6]])

```
then 
```
df.apply(max, args=[] axis=1)
```
applies the `max()` function on each row (`axis = 1`). Here, `args` is empty since `max` does not take any additional arguments.

The example below shows how to use `apply` when the function requires additional arguments. In this example, we apply a function that sums all the values of a row and adds to the sum an offset (2 by default)



In [67]:
# EXAMPLE CODE PROVIDED -- DO NOT REMOVE
def add_val_to_sum(x, offset=2):
    return x.sum() + offset
    
df = pd.DataFrame([[1,2,3], [4,5,6]])

print("The sum of rows + an offset of 5 is:")
print(df.apply(add_val_to_sum, args=[5], axis=1))

print("The sum of rows + an offset of 10 is:")
print(df.apply(add_val_to_sum, args=[10], axis=1))
   

The sum of rows + an offset of 5 is:
0    11
1    20
dtype: int64
The sum of rows + an offset of 10 is:
0    16
1    25
dtype: int64


In [80]:
### Note: I am going by what is stated on the Slack general channel. I am choosing my own subset to apply 'hash_on_subset'
my_column_names = pd.read_csv('data/all_kmers_10k.csv', nrows = 1).columns.tolist()
my_cols = ['AAAAAAAA', 'CCCCCCCC', 'GGGGGGGG', 'TTTTTTTT']
my_cols.insert(0, 'Unnamed: 0')
my_hash_vals = {}
blocks = pd.read_csv('data/all_kmers_10k.csv', chunksize = 100, usecols = my_cols)
for block in blocks:
    my_df = pd.DataFrame(block)
    my_values = my_df['Unnamed: 0'].to_list()
    my_keys = my_df.apply(hash_on_subset, args = [my_cols], axis = 1).to_list()
    for key in my_keys:
        if (key in my_hash_vals):
            my_hash_vals[key].append(my_values[my_keys.index(key)])
        else:
            my_hash_vals[key] = my_values[my_keys.index(key)]
print(my_hash_vals)

{-6069811906817578283: 'MT079851.1', -8612835751743951059: 'OD977007.1', -1774718700819342553: 'MW796028.1', 4169069491974424456: 'MW559193.1', -7381810108493157953: 'OU421224.1', 2978346575895000910: 'OU031839.1', -511746975245766441: 'MW030232.1', 5333124369033721975: 'MZ553971.1', 3145775866137227267: 'MZ850453.1', 3052048067001903704: 'MZ371569.1', 6288804848901968574: 'MZ001071.1', 6299822425847572847: 'MZ592007.1', 7778726948246322049: 'MZ002141.1', -5301984667039078866: 'MW908942.1', 4122125998123572570: 'MZ002723.1', -6155413997669135676: 'MZ069458.1', 7663914976110502467: 'MZ141510.1', 8016028443068801862: 'OU031807.1', 3160661810350988196: 'OU030348.1', 5407970613560875805: 'OD984828.1', 2863798129970320356: 'OD977249.1', -3889158374506224497: 'OD968949.1', -2771397008126306957: 'OD961244.1', -4621393487110258756: 'OD951702.1', -6617130752387247050: 'OD949720.1', -2711575798992226307: 'OD947749.1', 5501002670597226649: 'OD944351.1', 842524204384633022: 'OD941005.1', -93584816

### Q.13


Use `apply()` to apply `hash_subset` and compute the hash values for all the rows of `all_kmers_df` over `subset_kmers`

* Create a dict by parsing the results to group sequences that yield the same hash under the same bins. Each key in the dict should be a key and each value is a list of sequences that have the same value.

For example, in the dictionary below, X and Y have the same hash value (123456) over a given subset of kmers, whereas Z has a different hash over the same subsets.

```
{"123456": [X,Y], "654321": [Z]}
```


In [71]:
column_names = pd.read_csv('data/all_kmers_10k.csv', nrows = 1).columns.tolist()
cols = subset_kmers
cols.insert(0, 'Unnamed: 0')
hash_vals = {}
blocks = pd.read_csv('data/all_kmers_10k.csv', chunksize = 100, usecols = cols)
for block in blocks:
    df = pd.DataFrame(block)
    values = df['Unnamed: 0'].to_list()
    keys = df.apply(hash_on_subset, args = [subset_kmers], axis = 1).to_list()
    for key in keys:
        if (key in hash_vals):
            hash_vals[key].append(values[keys.index(key)])
        else:
            hash_vals[key] = values[keys.index(key)]
print(hash_vals)

{-7978453262802551738: 'MT079851.1', 6595557853672850001: 'OD977007.1', 8575804864725610453: 'MW796028.1', -8626882303312204937: 'MW559193.1', 6771445298618867736: 'OU421224.1', -3111370992851918878: 'OU031839.1', 9167971182009255164: 'MW030232.1', -2062498592165897048: 'MZ553971.1', -8356750340337191144: 'MZ850453.1', 4832564134686059015: 'MZ371569.1', -8664886465036966726: 'MZ001071.1', -8517309751956089197: 'MZ592007.1', -5645193318035659878: 'MZ002141.1', 1478492423202816217: 'MW908942.1', 2878701589560395760: 'MZ002723.1', 8409395941319318819: 'MZ069458.1', -6800690585557678522: 'MZ141510.1', -4036081156736392838: 'OU031807.1', 4618015851159411648: 'OU030348.1', 3272776640598969075: 'OD984828.1', -7638842924876345245: 'OD977249.1', 8107782103906927667: 'OD968949.1', 7650219056001183624: 'OD961244.1', 8891191245606455104: 'OD951702.1', 9125669031471521947: 'OD949720.1', 503851141818138447: 'OD947749.1', -372372310008860580: 'OD944351.1', -2286197122805550914: 'OD941005.1', 83958478

### Q. 15

Here we use the presence and absence of words, i.e., a vector of booleans, to encode a sequence. The problem with this approach is that it considers the sequences to be identical, even if their word counts differ substantially. For example, given the sequence `X`, `Y` and `Z` as follows
```
X = ATAGATAGATAGATAGATT
Y = ATAGATAGATAGATAGATT
Z = ATAGATTTTTTTTTTTTTT
```
With k =2, all three sequences mach on their vector of word presence/absense.

In [25]:
X = "ATAGATAGATAGATAGATT"
Y = "ATAGATAGATAGATAGATT"
Z = "ATAGATTTTTTTTTTTTTT"

counts_dict = {"X": get_kmer(X, k=2), "Y": get_kmer(Y, k=2), "Z": get_kmer(Z, k=2)}
pretty_print_counts(counts_dict)

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
X,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1
Y,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1
Z,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1


* Comparing these sequences based on word counts, X is much more similar to Y than it is to Z

In [26]:
def get_kmer_counts(X, k):
    DNA = ["A", "C", "G", "T"]
    words_size_k = ["".join(prod) for prod in product(*([DNA]*k))]
    counts = pd.Series([0 for _ in words_size_k], index = words_size_k)
    counts_words_in_x = Counter([X[i:i+k] for i in range(0, len(X)-k+1)])
    counts.update(counts_words_in_x)
    return counts    


counts_dict = {"X": get_kmer_counts(X, k=2), "Y": get_kmer_counts(Y, k=2), "Z": get_kmer_counts(Z, k=2)}
pretty_print_counts(counts_dict)

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
X,0,0,4,5,0,0,0,0,4,0,0,0,4,0,0,1
Y,0,0,4,5,0,0,0,0,4,0,0,0,4,0,0,1
Z,0,0,1,2,0,0,0,0,1,0,0,0,1,0,0,13


* Given an example (vectors) to justify why hashing is not ideal with counts.
  * Use any means you think are useful to illustrate your point (e.g.: figure, simulation (yes, please!))
 
* Describe how the random project approach discussed in class can help solve the issue discussed
  * Use code to illustrate how random projection works in the following example.
    * I.e., provide code to provide an example where `X` and `Y` are assigned to the same bin and `Y` is assigned to a different bin.
    * You can choose any vector values as needed 
 
```python
X = [1,2]
Y = [2,2]
Z = [5,1]
```

In [57]:
### FIRST QUESTION: Given an example (vectors) to justify why hashing is not ideal with counts ###
A = "AAACAGATCACCCGCTGAG"
B = "AAACAGATCACCCGCTGAT"
print("Consider the sequences: A = 'AAACAGATCACCCGCTGAG' and B = 'AAACAGATCACCCGCTGAT' that differ only by the last character.")
counts_dict_new = {"A": get_kmer_counts(A, k=2), "B": get_kmer_counts(B, k=2)}
# Using Jaccard Similarity
jaccard_new = jaccard(counts_dict_new["A"], counts_dict_new["B"])
print(f"Their Jaccard Similarity is: {jaccard_new}.")
# Using hash_on_subset
new_subset = ['AA', 'AC','CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'TC', 'TG']
hash_sub_A = hash_on_subset(counts_dict_new["A"], new_subset)
hash_sub_B = hash_on_subset(counts_dict_new["B"], new_subset)
print(f"The two sequence hash to the same bin over the subset: {new_subset}")
print(f"Their hash values over the mentioned aubset are: A hashes to {hash_sub_A} == B hashes to {hash_sub_B}")
# Using Counts
hash_A = hash(tuple(counts_dict_new["A"]))
hash_B = hash(tuple(counts_dict_new["B"]))
print(f"But notice what happens when we hash based on counts over the entire subset. The two sequences hash to very different bins:")
print(f"A hashed to {hash_A} while B hashes to {hash_B}")
print("This is why hashing is not ideal with counts! They do not recognize very similar sequences and are very sensitive to small differences!")
pretty_print_counts(counts_dict_new)

Consider the sequences: A = 'AAACAGATCACCCGCTGAG' and B = 'AAACAGATCACCCGCTGAT' that differ only by the last character.
Their Jaccard Similarity is: 0.7142857142857143.
The two sequence hash to the same bin over the subset: ['AA', 'AC', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'TC', 'TG']
Their hash values over the mentioned aubset are: A hashes to -6868156890634477238 == B hashes to -6868156890634477238
But notice what happens when we hash based on counts over the entire subset. The two sequences hash to very different bins:
A hashed to -2691947758777979673 while B hashes to 7071735223896907604
This is why hashing is not ideal with counts! They do not recognize very similar sequences and are very sensitive to small differences!


,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
A,2,2,2,1,2,2,1,1,2,1,0,0,0,1,1,0
B,2,2,1,2,2,2,1,1,2,1,0,0,0,1,1,0


In [81]:
### SECOND QUESTION: Describe how the random project approach discussed in class can help solve the issue discussed ###

My response: The idea behind random projections is that data points in n-dimensional space that are relatively close together are more similar to those that are far apart. To verify this, we use random projections. First we vectorize our data points and choose a 'random' vector. The random vector can be any vector we choose. The idea is: depending where on the random vetor our own vectors are projected - that is: the ratio of the projection of our vectors onto the random vector - will determine what bins our own vectors will fall into. This means that points that are close together in space will project realively close together on a random vector and will bin to the same this.

This is a much more reliable approach to count hashing because random projections allows a reasonable margin of error. This is described by the Johnson-Lindenstauss Lemma. Our margins are approximately 1 + epsilon.

To demonstrate
Consider the following vectors
X: (1,2)
Y: (2,2)
Projected on
Z: (5,1)

Code:

In [114]:
import math

point_A = [1,2]
point_B = [2,2]
point_C = [5, 1]
vector_D = [3, .5]
size_D = math.sqrt(vector_D[0]**2 + vector_D[1]**2)
A_dot_D = point_A[0]*vector_D[0] + point_A[1]*vector_D[1]
proj_A = A_dot_D / size_D
B_dot_D = point_B[0]*vector_D[0] + point_B[1]*vector_D[1]
proj_B = B_dot_D / size_D
C_dot_D = point_C[0]*vector_D[0] + point_C[1]*vector_D[1]
proj_C = C_dot_D / size_D
print(f"Point A bins to: {math.ceil(proj_A/size_D)} and Point B bins to: {math.ceil(proj_B/size_D)}")
print(f"While point C bins to: {math.ceil(proj_C/size_D)} if we consider a vector: D = (3, .5)\n")
print(f"Notice that this is because the projection of A onto D: {proj_A} and the projection of B onto D: {proj_B}")
print(f"while the projection of C onto D: {proj_C}")
print("Notice that although point A and point B do not have the same projection value, they still bin to the same place")
print("This is one benefit of random projections: we are able to identify similar items with an ample margin of difference (unlike count hashing)")

Point A bins to: 1 and Point B bins to: 1
While point C bins to: 2 if we consider a vector: D = (3, .5)

Notice that this is because the projection of A onto D: 1.3151918984428583 and the projection of B onto D: 2.301585822275002
while the projection of C onto D: 5.0963686064660765
Notice that although point A and point B do not have the same projection value, they still bin to the same place
This is one benefit of random projections: we are able to identify similar items with an ample margin of difference (unlike count hashing)
